## Scrape Athlete List

Pulls list of athletes and their schools.



In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from time import sleep

Enter name of event and URL to scrape

In [17]:

Event= "&Event=1"
Event_file = 'men100'

In [12]:
district_ranges = [(115902, 115936), 
                   (97851, 97885), 
                   (89004, 89038), 
                   (79755, 79789),
                   (70553, 70586),
                   (59529, 59562),
                   (49416, 49448),
                   (43150, 43182),
                   (32345, 32377), 
                   (28032, 28064), 
                   (17269, 17301),
                   (12573, 12604),
                   (9316, 9347)]
                                    

In [78]:
def get_year(Div,Div_id,Event):
    id_key_string =     "../Athlete.aspx?AID="
    school_key_string = "../School.aspx?SchoolID="

    athlete_list_url="https://www.athletic.net/TrackAndField/Division/Event.aspx?DivID="+Div_id+Event

    response = requests.get(athlete_list_url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    cols = ['ID','Name','School_ID','School','Division']
    athlete_list = pd.DataFrame(columns=cols)

    tables=soup.find_all("table")
    
    title = soup.find("title")
    t = str(title).split()
    if "Men's" in t:
        end = t.index("Men's")
    else:    
        end = t.index("Women's")
        
    conf = ' '.join(t[1:end])
     
    for i in tables:
        for row in i:
            try:
                str_row = str(row)
                id_index = str_row.find(id_key_string)
                
                if id_index != -1:
                    
                    str_row = str_row[id_index+20:]            
                    id_str = str_row[0:8]
                    id_str = id_str.replace('\"',"")
                    id_str = id_str.replace('>','')
                    
                    name_index = str_row.find(">") + 1
                    str_row = str_row[name_index:]
                    name_index = str_row.find("<")
                    Name = str_row[0:name_index]
                    
                    school_index = str_row.find(school_key_string)
                    str_row = str_row[school_index+24:]
                    school_index = str_row.find('\">')
                    school_id = int(str_row[0:school_index]) 
                    
                    str_row = str_row[school_index+2:]
                    school_index = str_row.find("<")
                    school_name = str_row[0:school_index] 
                    
                    if id_str.isdigit():
                        ID = int(id_str)
                        athlete_list = athlete_list.append({'ID': ID, 'Name': Name, 'School_ID': school_id, 'School': school_name, 'Division': conf}, ignore_index=True)
            except ValueError as ve:
                print('Encountered error')
    athlete_list = athlete_list.set_index(['ID'])
    return athlete_list


In [79]:
#Test on one event
n = get_year(0, str(115902), Event)
n

,Name,School_ID,School,Division
ID,,,,
10795088,Greg Matzelle,20661,Binghamton,America East
8991846,Thomas Cooper,20661,Binghamton,America East
17799642,Anthony Urbancik,21430,Stony Brook,America East
17848611,Taiki Hirooka,20661,Binghamton,America East
16200457,Shamar Barnes,21079,Maryland-Baltimore County,America East
14309640,Ben Cotton,21062,Maine,America East
17748468,Jaeden Burke,21079,Maryland-Baltimore County,America East
16218306,Michael Hermance,20591,Albany,America East
14390854,Jowell Dixon,20591,Albany,America East


In [84]:
all_athlete_list.tail(20)

,Name,School_ID,School,Division
ID,,,,
2448017,George Sanchez,21335,Santa Clara,West Coast (WCC)
1231850,Eric Chandler,21265,Portland,West Coast (WCC)
2459461,Jason Thomas,21488,UC San Diego,West Coast (WCC)
824749,Andrew Van Straaten,21488,UC San Diego,West Coast (WCC)
2749272,Greg Palmer,21335,Santa Clara,West Coast (WCC)
343686,Julius Jones,21053,Loyola Marymount,West Coast (WCC)
728818,Arturo Bimrose,21265,Portland,West Coast (WCC)
662682,Mike Coleman,21048,Louisiana Tech,Western (WAC)
3311164,Silas Pimentel,21496,Utah State,Western (WAC)


In [87]:
lst = [('&Event=19', 'women100'),('&Event=80', 'women5000'), 
       ('&Event=79', 'men5000'), ('&Event=18', 'menTJ'), ('&Event=36', 'womenTJ'),
      ('&Event=34', 'womenPV'), ('&Event=16', 'menPV')]

for l in lst:
    Event = l[0]
    Event_file = l[1]
    print(l)
    all_athlete_list = pd.DataFrame()
    
    for district_range in district_ranges:
        for d in range(district_range[0], district_range[1] + 1):
            new_list = get_year(0,str(d),Event)
            all_athlete_list = all_athlete_list.append(new_list)
            
    print(all_athlete_list.head())
    all_athlete_list.to_csv('../athlete_list_'+Event_file+'.csv')

('&Event=19', 'women100')
Encountered error invalid literal for int() with base 10: '/a></td><td class="text-nowrap hidden-print' for athlete
Brianna Rollins-McNeal
Encountered error invalid literal for int() with base 10: 'd><td class="text-nowrap hidden-print' for athlete
Gabrielle Thomas
Encountered error invalid literal for int() with base 10: 'td><td class="text-nowrap hidden-print' for athlete
Vanessa Spizewski
Encountered error invalid literal for int() with base 10: '<td class="text-nowrap hidden-print' for athlete
Deajah Stevens
Encountered error invalid literal for int() with base 10: '></td><td class="text-nowrap hidden-print' for athlete
Theresa (Raub) Kroll
                       Name School_ID       School      Division
ID                                                              
14393372       Dana Cerbone     21430  Stony Brook  America East
14307346  Dominique Jackson     20661   Binghamton  America East
17799578       Jaelyn Davis     21430  Stony Brook  America E

Encountered error invalid literal for int() with base 10: '/td><td class="text-nowrap hidden-print' for athlete
Gardenia Centanaro
Encountered error invalid literal for int() with base 10: 'td class="text-nowrap hidden-print' for athlete
Sarah Hopkins
Encountered error invalid literal for int() with base 10: '<td class="text-nowrap hidden-print' for athlete
Bonnie Draxler
Encountered error invalid literal for int() with base 10: '></td><td class="text-nowrap hidden-print' for athlete
Theresa (Raub) Kroll
Encountered error invalid literal for int() with base 10: 'd class="text-nowrap hidden-print' for athlete
Julia Newton
                       Name School_ID         School      Division
ID                                                                
12540940  Mallory Prelewicz     20661     Binghamton  America East
14308652  Samantha Martinez     20591         Albany  America East
12540793     Jordan Hoffman     21176  New Hampshire  America East
16204866       Taylor Nydam     2117